# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096962


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:01<00:25,  1.10it/s]

Rendering models:  10%|█         | 3/30 [00:04<00:38,  1.42s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:46,  1.78s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:32,  1.29s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:12<00:00,  5.17it/s]

Rendering models:  93%|█████████▎| 28/30 [00:12<00:00,  4.89it/s]

Rendering models:  97%|█████████▋| 29/30 [00:12<00:00,  3.95it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  4.47it/s]

not-logged-in-3b0b929f4475fd276561    0.003346
matiasd123                            0.000275
Jnursssmith5263                       0.000327
DannyU                                0.000355
preshuss1                             0.000163
Quarkgc                               0.000090
jnarayanbvg                           0.014963
Jonas_Cross                           0.000229
not-logged-in-d78ab5a3057206e8280b    0.004692
not-logged-in-2f77ea516eada33edf99    0.000107
Mr_DMan                               0.000235
not-logged-in-7e969a324309f376dde8    0.000413
carolynjoy                            0.001116
clairedeu                             0.000247
not-logged-in-214814e52004b42491a4    0.001791
froggler                              0.018447
acapirala                             0.000716
not-logged-in-2c4a9bf7eca3b2d9e58f    1.200794
not-logged-in-f47183e1b336e5121b01    0.000355
Lavadude                              0.022063
jordanrushworth                       0.000383
gwesthof     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())